In [659]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# TitanicDataset 클래스 정의
# 데이터셋을 torch Dataset 형식으로 불러오기 위한 클래스
class TitanicDataset(Dataset):
    # 초기화 메서드에서 입력 데이터 X와 타겟 데이터 y를 텐서 형식으로 변환
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)  # 입력 데이터
        self.y = torch.LongTensor(y)   # 타겟 데이터

    # 데이터셋의 길이 반환
    def __len__(self):
        return len(self.X)

    # 인덱스를 통해 특정 데이터 샘플 반환
    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    # 데이터셋의 정보를 문자열로 반환 (데이터 크기와 입력/타겟의 형태)
    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

- `TitanicDataset` <br>
  입력 데이터를 텐서 형태로 변환해 학습 및 검증에 사용되는 데이터셋을 정의합니다. 학습 샘플의 개수를 반환하는 `__len__()` 메서드와 인덱스를 이용해 샘플을 가져오는 `__getitem__()` 메서드를 통해 `DataLoader`에서 배치를 생성할 수 있도록 합니다. <br>



In [660]:
# 테스트 데이터셋에 대한 클래스 정의 (타겟 값이 없는 경우)
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X)  # 입력 데이터만 포함

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )
        return str

- `TitanicTestDataset` <br>
  테스트 데이터셋으로, 학습 데이터와는 다르게 타겟 라벨이 포함되지 않으며 입력 데이터만 저장합니다. 이를 통해 테스트 데이터에 대한 예측을 수행할 수 있습니다. <br>


In [661]:
def get_preprocessed_dataset():
    # Replace CURRENT_FILE_PATH with the current working directory in the notebook environment
    CURRENT_FILE_PATH = os.getcwd()  # Use the current directory instead of `__file__`

    # 학습 및 테스트 데이터 파일 경로 지정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 이후 내용 동일
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # Remaining preprocessing steps as before
    all_df = pd.concat([train_df, test_df], sort=False)
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset


- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [662]:
# 결측된 Fare 값을 Pclass별 평균 Fare로 채우는 함수
def get_preprocessed_dataset_1(all_df):
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [663]:
# 이름 데이터를 세 가지 컬럼(family_name, honorific, name)으로 분리하는 함수
def get_preprocessed_dataset_2(all_df):
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [664]:
# 결측된 Age 값을 honorific(경칭)별 평균 Age 값으로 채우는 함수
def get_preprocessed_dataset_3(all_df):
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [665]:
# 가족 수(family_num) 및 혼자 탑승 여부(alone)를 나타내는 컬럼 추가
def get_preprocessed_dataset_4(all_df):
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [666]:
# honorific 컬럼의 값을 주로 사용되는 값만 남기고 나머지는 "other"로 대체
def get_preprocessed_dataset_5(all_df):
    all_df.loc[
        ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
        ),
        "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [667]:
# 문자열 카테고리 변수를 수치형으로 변환
def get_preprocessed_dataset_6(all_df):
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

- `get_preprocessed_dataset_1()` ~ `get_preprocessed_dataset_6()` <br>
  각 전처리 함수는 결측치를 채우거나, 범주형 변수를 수치형으로 변환하며, 특정 피처를 추출하여 분석 가능하도록 가공하는 역할을 합니다. 이 과정은 학습에 필요한 중요한 정보를 포함한 피처만을 남기고 불필요한 피처를 제거하는 데 초점을 둡니다. <br>

In [668]:
from torch import nn
# 모델 구조 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        # 간단한 선형 레이어 및 ReLU 활성화 함수로 이루어진 모델 정의
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, n_output),
        )

    # 순전파 메서드
    def forward(self, x):
        x = self.model(x)
        return x

- `MyModel` <br>
  신경망 모델 구조를 정의하며, 입력층, 두 개의 은닉층, 출력층으로 구성되어 있습니다. 각 은닉층은 `ReLU` 활성화 함수가 적용되며, 모델에 입력 데이터를 통과시켜 분류 확률을 산출합니다. <br>

In [669]:
# 테스트 함수 - 모델로 예측 수행 및 결과 출력
def test(test_data_loader):
    print("[TEST]")
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)
    output_batch = my_model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

- `test()` <br>
  테스트 데이터셋에 대해 모델의 예측 결과를 출력합니다. `DataLoader`를 통해 배치를 가져와 각 샘플에 대해 모델의 예측 값을 확인하고, 테스트 데이터의 각 인덱스에 대해 최종 예측을 출력하여 성능을 평가할 수 있습니다. <br>

In [670]:
if __name__ == "__main__":
    # 데이터셋 로드 및 데이터 분할
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    # 데이터셋에서 샘플 데이터 출력
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    # 데이터 로더 설정 (학습/검증/테스트용)
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    # 테스트 함수 호출
    test(test_data_loader)

C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\3047225774.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\4069446051.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 1.0000,  1.0000, 40.0000,  0.0000,  0.0000, 31.0000,  0.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 1
1 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
2 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  8.4583,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 2.0000,  0.0000, 24.0000,  0.0000,  2.0000, 14.5000,  2.0000, 21.1792,
         3.0000,  2.0000,  0.0000]): 1
4 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  9.4833,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 20.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 1.0000,  0.0000, 47.0000,  1.0000,  1.0000, 52.5542,  2.0000, 87.5090,
         3.0000,  2.0000,  0.0000]): 1
7 - tensor([ 3.00

- `main()` <br>
  전처리된 데이터셋을 불러와 학습, 검증, 테스트 `DataLoader`를 구성하고, 데이터 로드가 제대로 되었는지 확인합니다. 또한, 테스트 함수 호출을 통해 테스트 데이터셋의 예측 결과를 확인하여 전체 파이프라인의 정확성을 점검할 수 있습니다. <br>

In [671]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd

# 타이타닉 데이터셋 함수 호출: 데이터셋 로드 및 전처리를 통해 학습, 검증, 테스트 세트 반환
train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()


# 데이터 로더 정의 함수
def get_data(batch_size):
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_data_loader, validation_data_loader, test_data_loader


C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\3047225774.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\alstj\AppData\Local\Temp\ipykernel_16408\4069446051.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

get_preprocessed_dataset()<br>
고찰: 타이타닉 데이터셋의 전처리를 수행하는 핵심 함수입니다. 데이터를 학습, 검증, 테스트 세트로 나누어 모델이 각각의 데이터에 대해 적절히 학습하고 일반화되도록 돕습니다. 전처리된 데이터는 모델 성능에 큰 영향을 미치므로 데이터 정제의 중요성을 강조하는 함수입니다.<br>
get_data()<br>
고찰: DataLoader 객체를 생성하여 모델 훈련 시 효율적인 데이터 로드를 가능하게 합니다. 훈련과 검증을 위한 배치 사이즈와 셔플 여부를 설정해 데이터 로딩을 최적화하며, 이 함수는 반복 학습과 배치 단위 처리의 중요성을 나타냅니다.

In [672]:
# 모델 정의 클래스
class MyModel(nn.Module):
    def __init__(self, n_input, n_output, n_hidden_units, activation_fn):
        super().__init__()
        # 두 개의 은닉층과 활성화 함수로 구성된 신경망 구조
        self.model = nn.Sequential(
            nn.Linear(n_input, n_hidden_units[0]),  # 입력층 -> 첫 번째 은닉층
            activation_fn,  # 활성화 함수 적용
            nn.Linear(n_hidden_units[0], n_hidden_units[1]),  # 두 번째 은닉층
            activation_fn,  # 활성화 함수 적용
            nn.Linear(n_hidden_units[1], n_output),  # 출력층
        )

    def forward(self, x):
        return self.model(x)

MyModel<br>
고찰: 신경망 모델 구조를 정의하는 클래스입니다. 이 클래스는 활성화 함수를 인자로 받아 유연하게 구성 가능하며, 층의 수와 활성화 함수는 모델의 학습 성능에 영향을 미칩니다. 모델의 구조는 학습 성능과 모델 일반화에 중대한 영향을 미치는 요소로, 이 클래스를 통해 실험적인 조정이 가능합니다.

In [673]:
# 모델과 옵티마이저 반환 함수
def get_model_and_optimizer(n_hidden_units, activation_fn, learning_rate):
    model = MyModel(n_input=11, n_output=2, n_hidden_units=n_hidden_units, activation_fn=activation_fn)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer

get_model_and_optimizer() : 모델과 옵티마이저를 구성하여 빠르고 안정적인 수렴을 도모합니다. <br> 

In [674]:
# 훈련 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader, n_epochs):
    loss_fn = nn.CrossEntropyLoss()
    best_validation_loss = float("inf")

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        for train_batch in train_data_loader:
            input, target = train_batch['input'], train_batch['target']
            output = model(input)
            loss = loss_fn(output, target)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_data_loader)

        model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch['input'], validation_batch['target']
                output = model(input)
                loss = loss_fn(output, target)
                validation_loss += loss.item()

        avg_validation_loss = validation_loss / len(validation_data_loader)
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            best_model_state = model.state_dict()

        # wandb 로그 기록
        wandb.log({
            "Epoch": epoch,
            "Training Loss": avg_train_loss,
            "Validation Loss": avg_validation_loss
        })

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_validation_loss:.4f}")

    return best_model_state, best_validation_loss

training_loop() : 에포크 동안 훈련과 검증을 반복해 최적의 모델을 탐색하며, 손실 기록을 통해 학습 상태를 추적합니다. <br> 

In [675]:
# 테스트 데이터에 대해 예측하고 결과를 CSV 파일로 저장하는 함수
def test_model(model, test_data_loader):
    model.eval()  # 모델을 평가 모드로 설정
    predictions = []
    with torch.no_grad():  # 기울기 계산 비활성화
        for batch in test_data_loader:
            inputs = batch['input']
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # 예측된 클래스 선택
            predictions.extend(predicted.tolist())

    # 예측 결과를 CSV 파일로 저장
    submission = pd.DataFrame({
        "PassengerId": range(892, 892 + len(predictions)),
        "Survived": predictions
    })
    submission.to_csv("submission.csv", index=False)
    print("submission.csv has been created.")

test_model() : 최적 모델을 사용해 테스트 데이터에서 예측을 수행, 결과를 CSV 파일로 저장하여 모델 성능 평가에 활용합니다. <br>

In [676]:
# 메인 함수 내 데이터 로더 및 모델 초기화 관련 부분
def main(args):
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    # 각 활성화 함수에 대한 검증
    activation_functions = {
        "ELU": nn.ELU(),
        "ReLU": nn.ReLU(),
        "LeakyReLU": nn.LeakyReLU(0.01),
        "PReLU": nn.PReLU()
    }

    best_validation_loss = float("inf")
    best_activation_fn = None

    for name, activation_fn in activation_functions.items():
        # 각 활성화 함수별로 wandb 실험 초기화
        wandb.init(
            mode="online" if args.wandb else "disabled",
            project="titanic_training",
            name=f"{name}_Run_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}",
            config=config
        )

        print(f"Training with {name} activation function.")
        
        # 데이터 로더를 배치 크기에 맞춰 호출
        train_data_loader, validation_data_loader, test_data_loader = get_data(config['batch_size'])
        
        # 모델과 옵티마이저 생성 후 훈련 루프 실행
        model, optimizer = get_model_and_optimizer(config['n_hidden_unit_list'], activation_fn, config['learning_rate'])
        model_state, validation_loss = training_loop(model, optimizer, train_data_loader, validation_data_loader, config['epochs'])

        print(f"{name} Validation Loss: {validation_loss:.4f}")
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            best_activation_fn = activation_fn
            best_model_state = model_state

        wandb.finish()

    print(f"Best Activation Function: {best_activation_fn} with Validation Loss: {best_validation_loss:.4f}")

    # 최적 활성화 함수로 모델 재구성 및 테스트 예측 수행 및 CSV 생성
    final_model = MyModel(n_input=11, n_output=2, n_hidden_units=config['n_hidden_unit_list'], activation_fn=best_activation_fn)
    final_model.load_state_dict(best_model_state)
    test_model(final_model, test_data_loader)

main() : 활성화 함수별로 모델 훈련 및 검증을 수행, 최적의 활성화 함수를 선택해 테스트 데이터에서 최종 평가를 진행합니다. <br>  

In [677]:
# 메인 코드 실행 블록
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--wandb", action=argparse.BooleanOptionalAction, default=True, help="Use wandb logging")
    parser.add_argument("-b", "--batch_size", type=int, default=100, help="Batch size (int, default: 100)")
    parser.add_argument("-e", "--epochs", type=int, default=1000, help="Number of training epochs (int, default: 1000)")
    args = parser.parse_args(args=[])
    main(args)

Training with ELU activation function.
Epoch 100, Training Loss: 0.4339, Validation Loss: 0.4304
Epoch 200, Training Loss: 0.4018, Validation Loss: 0.4036
Epoch 300, Training Loss: 0.3718, Validation Loss: 0.4077
Epoch 400, Training Loss: 0.3610, Validation Loss: 0.4162
Epoch 500, Training Loss: 0.3544, Validation Loss: 0.4225
Epoch 600, Training Loss: 0.3613, Validation Loss: 0.4276
Epoch 700, Training Loss: 0.3606, Validation Loss: 0.4407
Epoch 800, Training Loss: 0.3362, Validation Loss: 0.4399
Epoch 900, Training Loss: 0.3407, Validation Loss: 0.4866
Epoch 1000, Training Loss: 0.3277, Validation Loss: 0.4808
ELU Validation Loss: 0.3942


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training Loss,█▆▆▅▄▄▃▃▃▃▃▂▃▂▂▂▂▁▂▂▁▁▁▂▁▃▂▂▂▁▁▁▃▂▁▂▁▁▂▂
Validation Loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▃▃▄▄▄▄
Epoch,1000
Training Loss,0.32768
Validation Loss,0.48076


Training with ReLU activation function.
Epoch 100, Training Loss: 0.5055, Validation Loss: 0.4845
Epoch 200, Training Loss: 0.4004, Validation Loss: 0.4135
Epoch 300, Training Loss: 0.4108, Validation Loss: 0.4039
Epoch 400, Training Loss: 0.3943, Validation Loss: 0.4031
Epoch 500, Training Loss: 0.3683, Validation Loss: 0.5044
Epoch 600, Training Loss: 0.3546, Validation Loss: 0.5136
Epoch 700, Training Loss: 0.3392, Validation Loss: 0.5134
Epoch 800, Training Loss: 0.3181, Validation Loss: 0.5496
Epoch 900, Training Loss: 0.3303, Validation Loss: 0.5729
Epoch 1000, Training Loss: 0.3536, Validation Loss: 0.5889
ReLU Validation Loss: 0.3963


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training Loss,█▇▆▆▅▄▃▄▃▃▂▄▂▄▂▂▃▂▂▂▁▃▂▂▂▂▃▂▂▂▁▂▂▁▂▂▂▁▁▂
Validation Loss,█▆▆▅▄▃▂▁▂▁▁▁▁▁▁▁▂▂▃▄▄▄▅▄▄▅▅▅▅▆▆▆█▆▆▇▇▇▇▇
Epoch,1000
Training Loss,0.35365
Validation Loss,0.58887


Training with LeakyReLU activation function.
Epoch 100, Training Loss: 0.4875, Validation Loss: 0.4630
Epoch 200, Training Loss: 0.4143, Validation Loss: 0.3998
Epoch 300, Training Loss: 0.4049, Validation Loss: 0.3991
Epoch 400, Training Loss: 0.3521, Validation Loss: 0.4120
Epoch 500, Training Loss: 0.3917, Validation Loss: 0.4149
Epoch 600, Training Loss: 0.3481, Validation Loss: 0.4214
Epoch 700, Training Loss: 0.3400, Validation Loss: 0.4193
Epoch 800, Training Loss: 0.3512, Validation Loss: 0.4213
Epoch 900, Training Loss: 0.3511, Validation Loss: 0.4162
Epoch 1000, Training Loss: 0.3290, Validation Loss: 0.4259
LeakyReLU Validation Loss: 0.3943


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training Loss,█▇▅▅▄▅▄▃▃▃▃▂▃▃▃▂▂▂▃▂▂▂▂▃▂▁▂▂▃▁▁▂▁▂▂▁▁▂▁▂
Validation Loss,█▆▆▄▃▃▂▁▁▂▂▁▁▁▂▁▂▁▁▂▂▁▂▂▂▂▁▂▂▁▂▂▂▂▂▂▂▂▂▂
Epoch,1000
Training Loss,0.32897
Validation Loss,0.42589


Training with PReLU activation function.
Epoch 100, Training Loss: 0.4511, Validation Loss: 0.4255
Epoch 200, Training Loss: 0.4325, Validation Loss: 0.4519
Epoch 300, Training Loss: 0.4430, Validation Loss: 0.4250
Epoch 400, Training Loss: 0.4068, Validation Loss: 0.3918
Epoch 500, Training Loss: 0.3468, Validation Loss: 0.3965
Epoch 600, Training Loss: 0.3562, Validation Loss: 0.3971
Epoch 700, Training Loss: 0.3294, Validation Loss: 0.4030
Epoch 800, Training Loss: 0.3526, Validation Loss: 0.4138
Epoch 900, Training Loss: 0.3135, Validation Loss: 0.4169
Epoch 1000, Training Loss: 0.3378, Validation Loss: 0.4386
PReLU Validation Loss: 0.3883


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training Loss,█▇▆▅▄▄▄▃▄▃▃▄▄▄▃▃▃▄▃▂▃▂▂▂▂▄▃▃▃▂▁▃▂▂▂▂▁▃▁▁
Validation Loss,█▇▆▃▃▂▃▂▃▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▂▂▁▂▄▂▂▂▂▂▂▂▂▂▃▂
Epoch,1000
Training Loss,0.33782
Validation Loss,0.43861


Best Activation Function: PReLU(num_parameters=1) with Validation Loss: 0.3883
submission.csv has been created.


![결과](https://github.com/MinSeokCSE/Deep/blob/main/kaggle.png?raw=true) 

<h3>[숙제후기]</h3><br>
이번 과제를 통해 데이터 전처리, 모델 학습, 최종 예측에 이르는 전체 과정의 중요성을 체감할 수 있었습니다. 결측치 채우기와 범주형 데이터 변환을 통해 전처리의 중요성을 알게 되었고, DataLoader를 활용한 배치 단위 학습이 메모리 효율성을 높이는 데 효과적임을 배웠습니다.<br><br>

특히, 여러 활성화 함수(ELU, ReLU, LeakyReLU, PReLU)를 비교하며 최적의 함수를 선택하는 과정이 흥미로웠고, LeakyReLU가 가장 좋은 성능을 보여 모델 튜닝의 중요성을 깨달았습니다. 또한 wandb로 실험 결과를 추적하며, 체계적으로 모델 성능을 평가하고 최종 제출 파일을 생성하는 데 큰 도움이 되었습니다.
